In [ ]:
# This file exports the background points for a given year as a shp file, using the feature collection and the costa rica borders

import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gbsc-gcp-lab-emordeca')



In [ ]:
# background points
# Load feature collection and borders
feature_collection = ee.FeatureCollection('projects/gbsc-gcp-lab-emordeca/assets/costa_rica/plantations/2018/CRpineapple2018')
borders = ee.FeatureCollection('FAO/GAUL/2015/level0')
costa_rica = borders.filter(ee.Filter.eq('ADM0_NAME', 'Costa Rica'))
costa_rica = ee.Feature(costa_rica.toList(costa_rica.size()).get(0))

# Convert the feature collection to a list
feature_list = feature_collection.toList(feature_collection.size()).slice(1)

# Iterate over the feature collection to subtract each feature from costa_rica
modified_costa_rica = costa_rica
for i in range(feature_list.size().getInfo()):
  feature = ee.Feature(feature_list.get(i))
  modified_costa_rica = modified_costa_rica.difference(feature)

# Generate random points within the costa_rica geometry
randomPoints = ee.FeatureCollection.randomPoints(costa_rica.geometry(), 3439)

# Define export parameters
export_params = {
    'collection': randomPoints,
    'description': '2018_non_pineapple_points',
    'fileFormat': 'SHP'
}

# Start the export task
task = ee.batch.Export.table.toDrive(**export_params)
task.start()